In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import uproot

### Lectura de los datos de entrada

In [2]:
# ubicación de los archivos de entrada y salida
root_folder = r'./datos_root/'
csv_folder = './datos_csv/'
filename = '1.root'

# lee el archivo root
file = uproot.open(root_folder+filename)
#file.keys()

In [3]:
# extrae los árboles del archivo root
ccd_trees = {}
for key in file.keys():
    name = key.split(";")[0]
    
    # descarta las llaves que no sean de muones
    if 'muonCCD' not in name:
        continue
    
    # almacena el primer árbol para cada CCD
    if name not in ccd_trees.keys():
        ccd_trees[name] = key;

list(ccd_trees.values())

['muonCCD1;15',
 'muonCCD2;13',
 'muonCCD3;13',
 'muonCCD4;12',
 'muonCCD5;11',
 'muonCCD6;10',
 'muonCCD7;9',
 'muonCCD8;9',
 'muonCCD9;8',
 'muonCCD10;8']

### Genera un catálogo con todos los impactos en el archivo ROOT

In [4]:
def generate_catalog(ccd):
    
    # construye un dataframe a partir del archivo de entrada
    df = pd.DataFrame(ccd.arrays(('DiffX', 'DiffY', 'eventNumber'), library="np"))

    # define las variables a extraer del dataframe
    features = [        
        # centroide
        'mean',
        
        # tamaño
        lambda x: np.max(x) - np.min(x),
        
        # coeficiente de asimetría
        lambda x: np.mean(x) - (np.max(x) + np.min(x))/2
    ]

    # extrae las variables seleccionadas
    df = df.groupby('eventNumber').agg(features)

    # asigna nombres legibles a las columnas
    catalog = pd.DataFrame({
        'centerX':df['DiffX']['mean'],
        'centerY':df['DiffY']['mean'],
        'sizeX':df['DiffX']['<lambda_0>'],
        'sizeY':df['DiffY']['<lambda_0>'],
        'asymX':df['DiffX']['<lambda_1>'],
        'asymY':df['DiffY']['<lambda_1>'],
    })
    
    # define un margen para excluir impactos cortados en los bordes
    margin = 500
    
    # selecciona los eventos que se dentro de los márgenes de cada CCD
    return catalog.loc[(margin-15000 < catalog['centerX']) & (catalog['centerX'] < 15000-margin) & 
                       (margin-34000 < catalog['centerY']) & (catalog['centerY'] < 26000-margin)]

In [5]:
%%time
catalogs = []

# ordena los CCD
name_key = sorted(ccd_trees.items(), key=lambda item: int(item[0].split('muonCCD')[1]))

# por cada CCD seleccionado
for name, key in name_key:
    
    # genera el catalogo
    df = generate_catalog(file[key])
    
    # inserta el numero del ccd
    df.insert(0, 'muonCCD', int(name.split('muonCCD')[1]))
    
    # almacena el resultado
    catalogs.append(df)
    print(name+'\t->\t[catalogs]')

# une los catalogos generados
filename_csv = csv_folder+filename+'.csv'
catalog = pd.concat(catalogs).sort_values('muonCCD')
catalog.to_csv(filename_csv)
print('[catalogs]\t->\t'+filename_csv)
print()

muonCCD1	->	[catalogs]
muonCCD2	->	[catalogs]
muonCCD3	->	[catalogs]
muonCCD4	->	[catalogs]
muonCCD5	->	[catalogs]
muonCCD6	->	[catalogs]
muonCCD7	->	[catalogs]
muonCCD8	->	[catalogs]
muonCCD9	->	[catalogs]
muonCCD10	->	[catalogs]
[catalogs]	->	./datos_csv/1.root.csv

CPU times: total: 1min 4s
Wall time: 1min 4s


### Visualiza el catálogo generado

In [6]:
catalog

,muonCCD,centerX,centerY,sizeX,sizeY,asymX,asymY
eventNumber,,,,,,,
15,1,-3701.897342,22563.553721,207.537701,212.857987,-31.277569,-32.923240
3337,1,-14469.547722,-31236.333861,156.780626,126.561988,-33.802897,5.992886
3329,1,-5046.802374,5690.118066,305.096291,222.780240,28.240876,-28.144576
3326,1,5549.341594,-9806.188557,130.048023,134.435952,-22.295212,11.608392
3314,1,2097.648522,-11652.124825,312.941495,145.603145,30.056139,27.051341
...,...,...,...,...,...,...,...
1568,10,1614.202784,-12437.717876,144.659781,274.466620,18.904786,43.530928
1589,10,4066.571850,-12290.154452,119.173413,128.597600,22.954914,6.806634
1598,10,-7683.928037,12937.357679,129.763422,143.874409,-18.488772,21.453253


In [7]:
# selecciona un evento al azar
catalog.loc[catalog.sample(1).index]

,muonCCD,centerX,centerY,sizeX,sizeY,asymX,asymY
eventNumber,,,,,,,
2999,1,-5064.380951,-24787.948185,138.397071,196.214007,-8.258058,27.366222
2999,2,-4828.516723,-25769.534584,141.371622,195.712613,-10.289676,26.402644
2999,3,-4592.623628,-26750.184052,149.998644,190.544550,-18.918302,24.526547
2999,4,-4357.006917,-27730.010092,153.612048,209.777870,-19.719708,35.233141
2999,5,-4122.057727,-28709.527848,138.349416,200.649948,-16.186671,31.662439
2999,6,-3886.550235,-29687.847937,135.229837,200.984673,-18.334476,31.450658
2999,7,-3650.671559,-30666.114262,127.057925,187.211596,-12.978349,24.630444
2999,8,-3414.889003,-31644.559883,131.909470,201.982133,-9.611577,29.576830
2999,9,-3178.073348,-32624.240468,142.449447,214.111648,-10.805808,34.915600
